In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from typing import Any # import Any from the typing module

# URL of the website
URL = "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"

def initialize_driver():
    # Initialize the Chrome driver
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    # Load the page
    driver.get(url)
    time.sleep(5) # wait for the page to load

# Functions to scrape bus routes
def scrape_bus_routes(driver):
    # Scrape bus routes
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Functions to scrape bus details
def scrape_bus_details(driver, url, route_name):
    # Scrape bus details
    try:
        driver.get(url)
        time.sleep(5) # wait for some 5 sec for the page to load

        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5) # wait for some 5 sec for the page to load

            #scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5) # wait for some 5 sec for the page to load

            # Find but details from the availbility
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPATH to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text.strip(),
                    "Bus_Type": bus_type_elements[i].text.strip(),
                    "Departing_Time": departing_time_elements[i].text.strip(),
                    "Duration": duration_elements[i].text.strip(),
                    "Reaching_Time": reaching_time_elements[i].text.strip(),
                    "Star_Rating": star_rating_elements[i].text.strip() if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text.strip(),
                    "Seat_Availability": seat_availability_elements[i].text.strip() if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details

        except Exception as e:
           print(f"Error occurred while scrapping bus details for {url}: {str(e)}")
           return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []
# List to hold all bus details
all_bus_details = []

# Main function to execute the scraping process
def scrap_all_pages():
    for page in range(1, 6): # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5) # wait for some 5 sec for the page to load
                
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            # Iterate over each bus route link and scrape details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                  all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrap_all_pages()

# Create a DataFrame from the scraped data
df_buses_2 = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_buses_2.to_csv('Aassam_bus_details.csv', index=False)

Error occurred while scrapping bus details for https://www.redbus.in/bus-tickets/guwahati-to-nagaon: Message: 
Stacktrace:
0   chromedriver                        0x00000001004ad208 cxxbridge1$str$ptr + 1927396
1   chromedriver                        0x00000001004a566c cxxbridge1$str$ptr + 1895752
2   chromedriver                        0x00000001000a0808 cxxbridge1$string$len + 89564
3   chromedriver                        0x00000001000e4bcc cxxbridge1$string$len + 369056
4   chromedriver                        0x000000010011e228 cxxbridge1$string$len + 604156
5   chromedriver                        0x00000001000d9698 cxxbridge1$string$len + 322668
6   chromedriver                        0x00000001000da310 cxxbridge1$string$len + 325860
7   chromedriver                        0x0000000100473e78 cxxbridge1$str$ptr + 1693012
8   chromedriver                        0x000000010047877c cxxbridge1$str$ptr + 1711704
9   chromedriver                        0x00000001004593ec cxxbridge1$str$pt

In [2]:
df_buses_2

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Himalayan Travels,NON AC Seater / Sleeper 2+1,23:40,04h 20m,04:00,1.7,INR 500,31 Seats available
1,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Yatra Travels,NON AC Seater/ Sleeper (2+1),20:00,04h 00m,00:00,3.7,350,9 Seats available
2,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,KIRAN TRAVELS,A/C Seater (2+1),20:40,03h 50m,00:30,3.1,500,11 Seats available
3,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Pranjit Travels (Under ASTC),A/C Seater / Sleeper (2+1),20:45,05h 45m,02:30,3.2,INR 500,18 Seats available
4,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Pranjit Travels (Under ASTC),Non AC Seater (2+1),21:15,04h 30m,01:45,3.2,INR 450,16 Seats available
...,...,...,...,...,...,...,...,...,...,...
87,North Lakhimpur to Nagaon (Assam),https://www.redbus.in/bus-tickets/north-lakhim...,KIRAN TRAVELS,A/C Seater (2+1),21:20,07h 10m,04:30,0,580,8 Seats available
88,Guwahati to Golaghat,https://www.redbus.in/bus-tickets/guwahati-to-...,RATNAGIRI TRANSPORT,Bharat Benz A/C Seater /Sleeper (2+1),21:31,06h 59m,04:30,4.0,INR 600,14 Seats available
89,Guwahati to Golaghat,https://www.redbus.in/bus-tickets/guwahati-to-...,NAMBOR TRANSPORT,Bharat Benz A/C Seater (2+1),21:00,07h 00m,04:00,4.1,INR 600,18 Seats available
90,Guwahati to Golaghat,https://www.redbus.in/bus-tickets/guwahati-to-...,NAMBOR TRANSPORT,NON A/C Seater (2+1),20:30,06h 45m,03:15,3.5,INR 500,19 Seats available
